## Webscraping

El siguiente notebook tiene como fin mostrar una de las herramientas de extracción de datos que consiste en obtener información de manera sistemática y ordenada de sitios web.
***



#### Librerías

El primer paso es llamar las librerías que se van a utilizar, las cuales dependen principalmente de los lenguajes utilizados (js, html, etc) en los portales web a explorar.

De manera introductoria usaremos un paquete de envío de protocolos **requests**, una librería para la organización de la información **pandas** y finalmente nuestra herramienta principal **bs4**
***

In [1]:
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd

#### Selección y exploración
El siguiente paso tiene que ver con la selección y exploración previa a realizar la extracción y un entendimiento de la página de la cual queremos extraer la información. Para este ejemplo usaremos el portal de la empresa de supermercados de grandes superficies Almacenes Éxito. https://www.exito.com. 
Aprovecharemos el buscador incorporado para realizar una busqueda específica, i.e. Celulares, de alguna categoría para recopilar todo el inventario con su precio y otros datos relevantes. 
***


#### Especificación URL y request
Luego de explorar el portal y ver qué nos interesa, podemos crear una variable especificando el url de interés  y continuar haciendo un request del contenido en dicha dirección.

In [2]:
url = "https://www.exito.com/browse/Tecnologia-Celulares_y_accesorios/_/N-2b5r"
html = get(url)
type(html)

requests.models.Response

Como podemos observar, 'html' es ahora un objeto de la clase models.Response que tiene distintos atributos. El que nos interesa es el atributo 'content' que contiene todo el código html del portal. Lo que sigue es convertir ese contenido en objetos manipulables en python mediante la librería de bs4.

#### Utilizando el parser y creando los objetos para manipular
***

In [3]:
soup = bs(html.content,'html.parser')
type(soup)

bs4.BeautifulSoup

Al tener un objeto de tipo bs4.BeautifulSoup podemos manipular y acceder sus diferentes atributos. Para eso debemos identificar qué datos queremos extraer y cuáles son las etiquetas de los objetos que los contienen. En este caso, sabemos que los productos están guardados en distintas etiquetas 'div' con varias clases, por eso usaremos la clase 'product' que generaliza las distintas clases para poder crear una lista que contenga todos los productos.

In [4]:
products = soup.find_all('div', class_ = "product")
len(products)

20

Para entender un poco como funciona la librería cogeremos uno de los elementos de este set de resultados obtenidos al encontrar los 'div' de clase 'product' en 'soup' y encontraremos las diferentes etiquetas que nos interesan de los datos. En este caso vamos a extraer los nombres de los productos, la marca de los fabricantes, el precio y el descuento si es que lo hay.

In [5]:
productTest = products[0]
productTest.a['title']

'Celular Motorola Moto Z2 Play 4gb 64gb 5.5 + Mod Bater A - www.exito.com'

In [6]:
productTest.find('span', class_="brand")

<span class="brand">MOTOROLA</span>

En este caso debemos acceder al atributo string de la etiqueta para extraer únicamente el texto de esta. 

In [7]:
productTest.find('span', class_="brand").string

'MOTOROLA'

In [8]:
productTest.find('span', class_="money").text

'1199940'

In [9]:
productTest.find('div', class_ = "discount-rate").text

'\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t40'

Ya tenemos las diferentes etiquetas que necesitamos para obtener los datos que queremos. El siguiente paso tiene que ver con recorrer todos los diferentes objetos dentro de 'products' y asignar los valores que encontremos a un vector que los guarde para usarlos más adelante. 

In [10]:
items = []
brands = []
prices = []
discounts = []

for product in products:
    item = product.a['title'][:-18]
    brand = product.find('span', class_="brand").string
    price = product.find('span', class_="money").string
    if(product.find('div', class_ = "discount-rate") != None):
        discount = product.find('div', class_ = "discount-rate").string
    else:
        discount = "-"
    items.append(item)
    brands.append(brand)
    prices.append(price)
    discounts.append(discount)        
    

In [11]:
df = pd.DataFrame({"Items": items,
                   "Brands": brands,
                   "Prices": prices,
                   "Discounts (%)": discounts})

df.head()

,Items,Brands,Prices,Discounts (%)
0,Celular Motorola Moto Z2 Play 4gb 64gb 5.5 + M...,MOTOROLA,1199940,40
1,Motorola One Blan,MOTOROLA,629932,51
2,Moto Z3 Play Azul Indi,MOTOROLA,1499925,25
3,Iphone 6 De 32 Gb Go,APPLE,949057,26
4,Celular Asus Zenfone 5z Negro 6gb 64gb 6.2 Pulgad,ASUS,1459119,47


Por último, exportaremos los datos encontrados a una tabla 'xlsx'.

In [12]:
df.to_excel("exito.xlsx")